In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats

# =========================================================
# 1. CONFIGURATION
# =========================================================
# Ensure 'a2' target is correct (brush_tail if avail, else contra)
TARGETS = [
    ("a2",   "brush_tail"),  
    ("pup1", "brush_tail"),
    ("pup2", "brush_tail") 
]

MAX_TRIALS = 8      # We only care about the first 8 repetitions
PRE_WINDOW = 0.5    
POST_WINDOW = 0.5   

# High Contrast Palette
CUSTOM_PALETTE = {
    "a2":   "#FA91A8",  
    "pup1": "#A8FA91",  
    "pup2": "#91FAC5"   
}

# =========================================================
# 2. PROCESSING
# =========================================================
trial_data = []

csv_path = Path("data/sessions_local.csv") if Path("data/sessions_local.csv").exists() else Path("data/sessions.csv")
meta = pd.read_csv(csv_path)

print(f"{'='*60}")
print(f"CALCULATING ADAPTATION (Trial 1 -> {MAX_TRIALS})")
print(f"{'='*60}")

for session, stim in TARGETS:
    row = meta.loc[(meta['session'] == session) & (meta['stim'] == stim)]
    if len(row) == 0: continue
    row = row.iloc[0]
    
    # Load Onsets
    onsets_path = Path(f"results/summaries/{session}_{stim}_onsets.txt")
    if not onsets_path.exists(): continue
    onsets = np.loadtxt(onsets_path)
    if onsets.ndim == 0: onsets = [onsets]
    
    # TRUNCATE to Max Trials (or keep all if less than max)
    current_trials = onsets[:MAX_TRIALS]
    
    # Load Spikes
    phy_dir = Path(row['phy_dir'])
    spike_times = np.load(phy_dir / 'spike_times.npy').flatten() / 30000.0
    spike_clusters = np.load(phy_dir / 'spike_clusters.npy').flatten()
    cluster_info = pd.read_csv(phy_dir / 'cluster_info.tsv', sep='\t')
    
    lbl = 'KSLabel' if 'KSLabel' in cluster_info.columns else 'group'
    valid_units = cluster_info[cluster_info[lbl].isin(['good', 'mua'])]
    
    # Layers
    def get_bound(col): return float(row[col]) if col in row and not pd.isna(row[col]) else None
    l1_bot  = get_bound('l1_end')
    l23_bot = get_bound('l23_end')
    l4_bot  = get_bound('l4_end')
    l6_bot  = get_bound('l6_end')
    if l1_bot is None: l1_bot = get_bound('s1_upper_um')

    for _, unit in valid_units.iterrows():
        uid = unit['cluster_id']
        depth = unit['depth']
        
        layer = "Other"
        if l23_bot is not None and l1_bot is not None and l23_bot <= depth <= l1_bot: layer = "L2/3"
        if l4_bot is not None and l23_bot is not None and l4_bot <= depth < l23_bot: layer = "L4"
        if l6_bot is not None and l4_bot is not None and l6_bot <= depth < l4_bot: layer = "L5/6"

        if layer not in ['L2/3', 'L4', 'L5/6']: continue
        
        u_spikes = spike_times[spike_clusters == uid]
        
        # --- TRIAL LOOP ---
        for i, t in enumerate(current_trials):
            trial_idx = i + 1 # 1-based index
            
            # Count Spikes
            c_pre = np.sum((u_spikes >= t - PRE_WINDOW) & (u_spikes < t))
            c_post = np.sum((u_spikes >= t) & (u_spikes < t + POST_WINDOW))
            
            # Rate (Hz)
            rate_pre = c_pre / PRE_WINDOW
            rate_post = c_post / POST_WINDOW
            
            # EVOKED RATE (Post - Pre)
            # We use this instead of Z-score to avoid dividing by zero variance in silent trials
            evoked = rate_post - rate_pre
            
            trial_data.append({
                'Animal': session,
                'Layer': layer,
                'Trial': trial_idx,
                'Evoked_Hz': evoked
            })

# =========================================================
# 3. PLOTTING
# =========================================================
if len(trial_data) == 0:
    print("❌ ERROR: No units found.")
else:
    df = pd.DataFrame(trial_data)
    
    # Setup Figure
    sns.set_style("ticks")
    sns.set_context("talk")
    
    # We use 'lmplot' (Linear Model Plot) to show the Trend Line + Data
    # scatter=False keeps it clean (just the mean line and confidence interval)
    g = sns.lmplot(
        data=df, x="Trial", y="Evoked_Hz", hue="Animal", col="Layer",
        col_order=['L4', 'L2/3', 'L5/6'], hue_order=['a2', 'pup1', 'pup2'],
        palette=CUSTOM_PALETTE, height=5, aspect=0.8,
        sharey=False, # Independent Y-axis (L4 is loud, L5 is quiet)
        scatter=False, # Hide the messy individual dots
        line_kws={'linewidth': 3, 'alpha': 0.8},
        ci=68 # Standard Error (narrower shading)
    )
    
    # Add dots for the actual Means (so you see the 'wiggles', not just the straight line)
    # We map a simple lineplot on top to show the actual trajectory
    for ax, (layer_name, sub_df) in zip(g.axes.flat, df.groupby('Layer')):
        sns.lineplot(data=sub_df, x="Trial", y="Evoked_Hz", hue="Animal", 
                     hue_order=['a2', 'pup1', 'pup2'],
                     palette=CUSTOM_PALETTE, ax=ax, legend=False,
                     linewidth=1, alpha=0.4, errorbar=None)
        
        ax.set_title(f"{layer_name}", fontweight='bold')
        ax.set_ylabel("Evoked Rate (Hz)" if layer_name=='L4' else "")
        ax.axhline(0, color='black', linestyle='--', alpha=0.3)
        
        # Grid lines for trials
        ax.set_xticks(range(1, MAX_TRIALS + 1))
        ax.grid(axis='x', linestyle=':', alpha=0.3)

    g.set_axis_labels("Stimulus Repetition", "")
    g.fig.suptitle("Habituation vs. Sensitization (First 8 Trials)", 
                   y=1.05, fontsize=18, fontweight='bold')
    
    plt.show()

    # =========================================================
    # 4. FIRST vs LAST TABLE
    # =========================================================
    print("\n" + "="*80)
    print(f"{'LAYER':<8} | {'ANIMAL':<8} | {'FIRST (T1)':<12} | {'LAST (T8/Max)':<12} | {'CHANGE':<10}")
    print("="*80)
    
    for layer in ['L4', 'L2/3', 'L5/6']:
        print(f">>> {layer}")
        subset = df[df['Layer'] == layer]
        
        for animal in ['a2', 'pup1', 'pup2']:
            an_data = subset[subset['Animal'] == animal]
            if len(an_data) == 0: continue
            
            # Get Mean of First Trial
            t1_mean = an_data[an_data['Trial'] == 1]['Evoked_Hz'].mean()
            
            # Get Mean of Last Available Trial
            max_t = an_data['Trial'].max()
            t_last_mean = an_data[an_data['Trial'] == max_t]['Evoked_Hz'].mean()
            
            # Calculate % Change or Delta
            delta = t_last_mean - t1_mean
            
            print(f"{'':<8} | {animal:<8} | {t1_mean:6.2f} Hz    | {t_last_mean:6.2f} Hz    | {delta:+6.2f} Hz")
        print("-" * 80)